# setup

In [1]:
!pip install openai tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [2]:
import openai
import tiktoken
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

client = openai.Client(api_key=api_key)

# helper function

In [3]:
def get_completion(prompt, model='gpt-3.5-turbo'):
  messages = [{'role': 'user', 'content': prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=1
  )

  return response.choices[0].message.content

# helper function in chat format

In [4]:
def get_completion_from_messages(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=temperature, # degree of randomness
      max_tokens=max_tokens # max number of tokens that model can output
  )

  return response.choices[0].message.content

# chain of thought reasoning

In [29]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products:
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information.

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [33]:
user_message = f"""
by how much is the Bluewave Chromebook more exprensive than the TechPro Desktop?
"""

# messages = [
#     dict(role='system', content=system_message),
#     dict(role='user', content=f'{delimiter}{user_message}{delimiter}')
# ]

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is comparing the prices of two specific products.
Step 2:#### Both products are available in the list of products provided.
Step 3:#### The assumption made by the user is that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is priced at $249.99. Therefore, the TechPro Desktop is actually more expensive than the BlueWave Chromebook.
Response to user:#### The BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop.


## hiding the internal monolouge from the user

In [36]:
try:
  final_response = response.split(delimiter)[-1].strip()
except Exception as e:
  final_response = "Sorry, I'm having trouble right now, please try again later"

print(final_response)

The BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop.


In [38]:
format_usr_msg = lambda x: f"{delimiter}{x}{delimiter}"

In [41]:
user_message = format_usr_msg('do you sell tvs?')
messages = [
    dict(role='system', content=system_message),
    dict(role='user', content=user_message)
]

response = get_completion_from_messages(messages)

try:
  final_response = response.split(delimiter)[-1].strip()
except Exception as e:
  final_response = "Sorry, I'm having trouble right now, please try again later"

print(final_response)

We currently do not sell TVs. Our store specializes in computers and laptops. If you have any questions about our available products in that category, feel free to ask!
